In [28]:
import numpy as np

## Funciones varias

In [29]:
def Puerta(name):
    if name == "H":
        return 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
    if name == "swap":
        return np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
    if name == "cx":
        return np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])
    
    raise ValueError("Puerta no definida")

In [65]:
def swaps(distancia):
    qubits = (0,distancia)
    num_qubits = qubits[1] - qubits[0] + 1
    swap = np.eye(2**num_qubits)
    mask1 = 1 << qubits[0]
    #anti_mask = ((1 << num_qubits) - 1) ^ mask1
    mask2 = 1 << qubits[1]

    for i in range(2**num_qubits):
        if (i & mask1) and ((i & mask2) == 0):
            swap[i,i] = 0
            swap[i-mask1+mask2,i-mask1+mask2] = 0
            swap[i,i-mask1+mask2] = 1
            swap[i-mask1+mask2,i] = 1

    return swap

In [66]:
swaps(2)

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [32]:
np.kron(Puerta("swap"),np.eye(2)) @ np.kron(np.eye(2),Puerta("swap")) @ np.kron(Puerta("swap"),np.eye(2))

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

## Clase QRegistry

In [60]:
class QRegistry:
    def __init__(self,nqubits):
        self.nqubits = nqubits
        self.estado = np.append(np.array([1],dtype=complex),np.array([0 for _ in range(2**self.nqubits - 1)],dtype=complex)).reshape(-1,1)

    def ket(self):
        return self.estado
    
    def bra(self):
        return np.conjugate(np.transpose(self.estado))
    
    def M_densidad(self):
        return np.dot(self.ket(),self.bra())
    
    def aplicar_puertaM(self, puerta, qubit):
        num_q_Puerta = int(np.log2(puerta.shape[0]))

        if qubit[0] < 0 or len(qubit) >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        elif num_q_Puerta > self.nqubits - len(qubit):
            raise ValueError("La puerta no cabe")
        
        cambios = None

        if len(qubit) > 1:
            posiciones = qubit + [j for j in range(self.nqubits) if j not in qubit]
            cambios = np.argsort(posiciones)
            # print(cambios, posiciones)
            
            for i in range(len(qubit)):
                if i != cambios[i]:
                    # print(i,cambios[i])
                    self.aplicar_puertaM(swaps(np.abs(i - cambios[i])),qubit=[i])

        operacion = np.kron(np.kron(np.eye(2**(self.nqubits - qubit[0] - num_q_Puerta)),puerta),np.eye(2**qubit[0]))

        self.estado = np.dot(operacion,self.ket())

        if cambios is not None:
            for i in range(len(qubit)):
                if i != cambios[i]:
                    # print(i,cambios[i])
                    self.aplicar_puertaM(swaps(np.abs(i - cambios[i])),qubit=[i])

        return self.estado
    
    def medir(self, qubit):
        if qubit < 0 or qubit >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2]
        # print(lista)
        r = np.random.rand()
        p = sum(np.absolute(self.estado[i,0])**2 for i in lista)

        if r < p:
            #print("Mide 1")
            listadg = [j for j in range(2**self.nqubits) if j//2**(qubit)%2 == 0]
            for i in listadg:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(p)
        else:
            #print("Mide 0")
            for i in lista:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(1-p)

        return int(r < p)


In [62]:
registro = QRegistry(4)
registro.aplicar_puertaM(Puerta("H"),qubit=[0])
# registro.aplicar_puertaM(swaps(1),qubit=[0])
registro.aplicar_puertaM(Puerta("cx"),qubit=[0,2])

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j]], dtype=complex256)

## Ejemplos de uso

In [ ]:
cx = Puerta("cx")
H = Puerta("H")

par_epr = QRegistry(2)
par_epr.aplicar_puertaM(H,[0])

par_epr.aplicar_puertaM(cx,[0])

display(par_epr.ket())

print(par_epr.medir(0))

print(par_epr.medir(1))

ValueError: La puerta no cabe

In [ ]:
registro = QRegistry(3)
registro.aplicar_puertaM(np.kron(hadamard,hadamard),1)
registro.ket()

array([[0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j]], dtype=complex256)

In [ ]:
registro = QRegistry(3)
hadamard = 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [ ]:
registro = QRegistry(3)
registro.medir(1)

0

In [ ]:
ket0 = QRegistry(2)
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

In [ ]:
ket0.bra()

array([[1.-0.j, 0.-0.j, 0.-0.j, 0.-0.j]])

In [ ]:
ket0.M_densidad()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [ ]:
ket0.aplicar_puerta(hadamard,qubit=[1])

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j]], dtype=complex256)

In [ ]:
ket0.medir(1)

0

In [ ]:
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [ ]:
registro = QRegistry(3)
registro.aplicar_puerta(hadamard,[0,1,2])
registro.ket()

array([[0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j],
       [0.35355339+0.j]], dtype=complex256)

In [ ]:
registro.ket().shape[0]

8

In [ ]:
registro = QRegistry(3)
registro.aplicar_puertaM(hadamard,[1])
registro.ket()

array([[0.70710678+0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j]], dtype=complex256)